# Import Library

In [1]:
import pandas as pd

import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('wordnet')

C:\Users\hrd\.conda\envs\py_TF2\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\hrd\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

# Import Dataset

In [2]:
data_wajib_pajak= pd.read_csv('WAJIB_PAJAK.csv', sep = '|')
data_faktur     = pd.read_csv('FAKTUR.csv', sep = '|')
data_obyek      = pd.read_csv('OBYEK.csv', sep = '|')
data_ekspor      = pd.read_csv('EKSPOR.csv', sep = '|')
data_impor      = pd.read_csv('IMPOR.csv', sep = '|')

In [3]:
data_faktur

,ID_FAKTUR,MASA_PAJAK,TAHUN_PAJAK,NPWP_PENJUAL,NPWP_PEMBELI
0,6937192014,6,2019,564779873166555,575601716960555
1,6276807069,4,2019,302335091102555,375739587354555
2,6154277515,8,2019,569277996756555,0
3,6025220056,8,2019,575175564886555,273381502886555
4,6121497811,3,2019,560977158960555,281561651852555
...,...,...,...,...,...
1465046,6191027245,3,2019,568881293583555,568556152509555
1465047,6031667342,8,2019,302335091102555,207251837359555
1465048,6265216175,4,2019,577417790584555,563041099060556
1465049,6962531219,1,2019,560977158960555,0


# Perhitungan PPN (10% * DPP * Jumlah Barang)

In [123]:
data_obyek['PPN barang'] = 0.1*(data_obyek['DPP'] * data_obyek['JUMLAH_BARANG'])

# Join Tabel Data Obyek dan Data Faktur

In [126]:
data_faktur=data_faktur[['ID_FAKTUR','MASA_PAJAK','NPWP_PENJUAL','NPWP_PEMBELI']]
dfinal_obyek = data_obyek.merge(data_faktur, on="ID_FAKTUR", how = 'inner')
dfinal_obyek.head()

,ID_OBJEK_FAKTUR,ID_FAKTUR,NAMA_BARANG,DPP,JUMLAH_BARANG,PPN barang,MASA_PAJAK,NPWP_PENJUAL,NPWP_PEMBELI
0,66806169458,6406426318,BIG 102 BC,81555.0,10.0,81555.0,1,562197926987555,0
1,3046030121,6984147727,Bearing Tan 6204 zz,604095.0,30.0,1812285.0,1,560540673980555,0
2,3046030129,6984147727,Gantungan Gembok,70205.0,2.0,14041.0,1,560540673980555,0
3,3046030127,6984147727,Handle Combined,24279.0,4.0,9711.6,1,560540673980555,0
4,3046030125,6984147727,Box Kunci 1 Muka,84727.0,4.0,33890.8,1,560540673980555,0


# Olah Data Impor & Ekspor

## Data ekspor

## Ambil masa pajak & Nilai FOBR

In [9]:
from tqdm import tqdm
masa_pajak_eks=[]
for i in tqdm(range(data_ekspor.shape[0])):
    string=data_ekspor.TGL_PEB.tolist()
    masa_pajak_eks.append(string[i][5:7])
    
data_ekspor['TGL_PEB'] = masa_pajak_eks
data_ekspor

100%|█████████████████████████████████████████████████████████████████████████| 209841/209841 [04:18<00:00, 811.77it/s]


,TGL_PEB,NPWP_EKSPORTIR,RINCIAN_BARANG,NILAI_FOBR
0,02,576764283960555,BAGIAN RACK DARI KAYU ALBASIA DAN MDF,766375.0
1,12,860034902531555,"LG,MD,SM RD BANANA BSKT NATXL,L,M,S BANANA BAR...",8951283.0
2,03,566816002068555,INDONESIAN HARDWOOD PLYWOOD8.5 MM X 910MM X 18...,35100184.2
3,03,564355431087555,FURNITURE ROTAN FULL,60453255.0
4,11,569771154678555,BENCH,69967070.0
...,...,...,...,...
209836,10,588686607527555,MEDIUM DENSITY FIBREBOARD (MDF),31919957.0
209837,09,562633433667555,RUBBERWOOD FINGER JOINT LAMINATED BOARD,891389358.0
209838,06,562797529066555,WALNUT ENGINEERED DOORS,34942355.0
209839,10,303800539063555,SAMPLE PLYWOOD,767125.0


In [44]:
data_eksportir=data_ekspor.groupby(['NPWP_EKSPORTIR','TGL_PEB'], as_index=False).agg({"NILAI_FOBR": "sum"})
data_eksportir

,NPWP_EKSPORTIR,TGL_PEB,NILAI_FOBR
0,110181345068555,01,6.094887e+09
1,110181345068555,02,5.416619e+08
2,110181345068555,03,2.151022e+08
3,110181345068555,04,6.129112e+08
4,110181345068555,05,3.669303e+08
...,...,...,...
6482,869853173197555,08,2.346003e+10
6483,869853173197555,09,2.339517e+10
6484,869853173197555,10,2.074601e+10
6485,869853173197555,11,1.559385e+10


In [51]:
list_barang = []
for i in range(data_eksportir.shape[0]):
    data_barang = data_ekspor.loc[(data_ekspor['NPWP_EKSPORTIR'] == data_eksportir['NPWP_EKSPORTIR'].iloc[i]) & (data_ekspor['TGL_PEB'] == data_eksportir['TGL_PEB'].iloc[i])]
    list_barang.append(data_barang.RINCIAN_BARANG.tolist())
    
len(list_barang)

6487

In [56]:
data_eksportir['RINCIAN_BARANG']=list_barang
data_eksportir.to_csv('data_ekspor_digrup_NPWP_TGL_PEB.csv')

## Data Impor

## Ambil masa pajak dan nilai CIF

In [57]:
data_impor

,TGL_PIB,NPWP_IMPORTIR,RINCIAN_BARANG,NILAI_CIF
0,2019-02-11,565157133509555,WOOD WORKING MACHINERY 50 TON COLD PRESS PLATE...,4448.27
1,2019-10-27,579687186987555,15M1 PE-STRIP COMPOUNT 12X5,1630.90
2,2019-01-13,296828244573555,PLATES 2000X1200X100MM RANPREX ML15E KP20224 S...,7343.89
3,2019-06-11,575775753957555,DECORATIVE PAPER SM 3008F (TG022B-1),6049.00
4,2019-04-12,563860185196555,"DECORATIVE PAPER, AMINO PAPER - 30G LEFKAS BRO...",7955.00
...,...,...,...,...
29684,2019-08-29,570740428167555,7/4 RECOVERY TRIM WESTERN RED CEDAR GREEN ROUG...,67889.31
29685,2019-03-31,569859009960555,HANDLE FA1039 C=160MM SATIN ALU,9957.76
29686,2019-11-11,360957432531555,PRESSURE GAUGE FOR HOT PRESS,61.00
29687,2019-05-23,568800962592555,"PLASTIC HANDLE FOR PAINT BRUSH SIZE 1-1/2"" PLA...",895.60


In [58]:
from tqdm import tqdm
masa_pajak_imp=[]
for i in tqdm(range(data_impor.shape[0])):
    string=data_impor.TGL_PIB.tolist()
    masa_pajak_imp.append(string[i][5:7])
    
data_impor['TGL_PIB'] = masa_pajak_imp
data_impor

100%|██████████████████████████████████████████████████████████████████████████| 29689/29689 [00:03<00:00, 9722.96it/s]


,TGL_PIB,NPWP_IMPORTIR,RINCIAN_BARANG,NILAI_CIF
0,02,565157133509555,WOOD WORKING MACHINERY 50 TON COLD PRESS PLATE...,4448.27
1,10,579687186987555,15M1 PE-STRIP COMPOUNT 12X5,1630.90
2,01,296828244573555,PLATES 2000X1200X100MM RANPREX ML15E KP20224 S...,7343.89
3,06,575775753957555,DECORATIVE PAPER SM 3008F (TG022B-1),6049.00
4,04,563860185196555,"DECORATIVE PAPER, AMINO PAPER - 30G LEFKAS BRO...",7955.00
...,...,...,...,...
29684,08,570740428167555,7/4 RECOVERY TRIM WESTERN RED CEDAR GREEN ROUG...,67889.31
29685,03,569859009960555,HANDLE FA1039 C=160MM SATIN ALU,9957.76
29686,11,360957432531555,PRESSURE GAUGE FOR HOT PRESS,61.00
29687,05,568800962592555,"PLASTIC HANDLE FOR PAINT BRUSH SIZE 1-1/2"" PLA...",895.60


In [59]:
data_importir=data_impor.groupby(['NPWP_IMPORTIR','TGL_PIB'], as_index=False).agg({"NILAI_CIF": "sum"})
data_importir

,NPWP_IMPORTIR,TGL_PIB,NILAI_CIF
0,116423255060555,03,78636.70
1,117589536068555,05,797230.00
2,118808499669555,12,69055.00
3,201273217088555,12,77833.00
4,206205770170555,03,1900846.00
...,...,...,...
1701,869853173197555,05,16322.00
1702,869853173197555,06,6232.10
1703,869853173197555,07,308415.56
1704,869853173197555,10,57018235.00


In [60]:
list_barang = []
for i in range(data_importir.shape[0]):
    data_barang = data_impor.loc[(data_impor['NPWP_IMPORTIR'] == data_importir['NPWP_IMPORTIR'].iloc[i]) & (data_impor['TGL_PIB'] == data_importir['TGL_PIB'].iloc[i])]
    list_barang.append(data_barang.RINCIAN_BARANG.tolist())
    
len(list_barang)

1706

In [61]:
data_importir['RINCIAN_BARANG']=list_barang
data_importir.to_csv('data_impor_digrup_NPWP_TGL_PIB.csv')

# Panggil dan olah data ekspor & Impor

# Data Impor & Ekspor sudah memiliki masa PIB & PEB

In [148]:
data_eksportir = pd.read_csv('data_ekspor_digrup_NPWP_TGL_PEB.csv')
data_importir = pd.read_csv('data_impor_digrup_NPWP_TGL_PIB.csv')

In [149]:
data_eksportir

,Unnamed: 0,NPWP_EKSPORTIR,TGL_PEB,NILAI_FOBR,RINCIAN_BARANG
0,0,110181345068555,1,6.094887e+09,['ALBASIA BARE CORE']
1,1,110181345068555,2,5.416619e+08,"['ALBASIA BARE CORE', 'ALBASIA BARE CORE']"
2,2,110181345068555,3,2.151022e+08,"['ALBASIA BARE CORE', 'ALBASIA BARE CORE']"
3,3,110181345068555,4,6.129112e+08,"['ALBASIA BARE CORE', 'ALBASIA BARE CORE']"
4,4,110181345068555,5,3.669303e+08,['ALBASIA BARE CORE']
...,...,...,...,...,...
6482,6482,869853173197555,8,2.346003e+10,"['INDONESIAN POLYESTER PLYWOOD', 'INDONESIAN P..."
6483,6483,869853173197555,9,2.339517e+10,"['INDONESIAN POLYESTER PLYWOOD', 'INDONESIAN P..."
6484,6484,869853173197555,10,2.074601e+10,"['INDONESIAN POLYESTER PLYWOOD', 'INDONESIAN P..."
6485,6485,869853173197555,11,1.559385e+10,"['INDONESIAN POLYESTER PLYWOOD', 'INDONESIAN P..."


In [150]:
data_importir

,Unnamed: 0,NPWP_IMPORTIR,TGL_PIB,NILAI_CIF,RINCIAN_BARANG
0,0,116423255060555,3,78636.70,['FSC 100 PCT CERTIFIED EUCALYPTUS RED GRANDIS...
1,1,117589536068555,5,797230.00,"['1.18/WOOLEN MAT/CUSHION PAD', '1.10/TRANSFER..."
2,2,118808499669555,12,69055.00,['MULTI-BLADE SAW MACHINE']
3,3,201273217088555,12,77833.00,['SZF50*27 THREE AXIS HAMMER MILL']
4,4,206205770170555,3,1900846.00,['TWO HEADS CALIBRATING AND POLISHING DUAL-PUR...
...,...,...,...,...,...
1701,1701,869853173197555,5,16322.00,['POLYESTER FILM FOR POLYESTER BOARD THICKNESS...
1702,1702,869853173197555,6,6232.10,['FINISH FOIL P 40 RIGOLETTO BORA 100918 6469 ...
1703,1703,869853173197555,7,308415.56,"['CLUTCH BRAKE BAIK DAN BARU', '300L SLURRY TA..."
1704,1704,869853173197555,10,57018235.00,['SPARE PARTS HAMMER CASTER TYPE: 420SAD100H13...


In [151]:
dfinal_obyek

,ID_OBJEK_FAKTUR,ID_FAKTUR,NAMA_BARANG,DPP,JUMLAH_BARANG,PPN barang,MASA_PAJAK,NPWP_PENJUAL,NPWP_PEMBELI
0,66806169458,6406426318,BIG 102 BC,81555.00,10.000,8.155500e+04,1,562197926987555,0
1,3046030121,6984147727,Bearing Tan 6204 zz,604095.00,30.000,1.812285e+06,1,560540673980555,0
2,3046030129,6984147727,Gantungan Gembok,70205.00,2.000,1.404100e+04,1,560540673980555,0
3,3046030127,6984147727,Handle Combined,24279.00,4.000,9.711600e+03,1,560540673980555,0
4,3046030125,6984147727,Box Kunci 1 Muka,84727.00,4.000,3.389080e+04,1,560540673980555,0
...,...,...,...,...,...,...,...,...,...
3630143,66695170713,6469551952,Blockboard Core 24K 24 x 960 x 1870 mm,6255355.00,17.234,1.078048e+07,12,303865310068555,867464278068555
3630144,66781327648,6474881364,KASUR PRINCE 180,8181818.20,4.000,3.272727e+06,12,569089169256555,0
3630145,66694924760,6460156817,FULL SET M/P FITTED JERSEY PU 200X200,6674545.33,2.000,1.334909e+06,12,560977158960555,510039378071555
3630146,66597008664,6347721868,FREIGHT FORWARDING SERVICES,8835555.00,1.000,8.835555e+05,12,568036144517555,569408433502555


In [152]:
#Import Data Objek
dfinal_obyek

,ID_OBJEK_FAKTUR,ID_FAKTUR,NAMA_BARANG,DPP,JUMLAH_BARANG,PPN barang,MASA_PAJAK,NPWP_PENJUAL,NPWP_PEMBELI
0,66806169458,6406426318,BIG 102 BC,81555.00,10.000,8.155500e+04,1,562197926987555,0
1,3046030121,6984147727,Bearing Tan 6204 zz,604095.00,30.000,1.812285e+06,1,560540673980555,0
2,3046030129,6984147727,Gantungan Gembok,70205.00,2.000,1.404100e+04,1,560540673980555,0
3,3046030127,6984147727,Handle Combined,24279.00,4.000,9.711600e+03,1,560540673980555,0
4,3046030125,6984147727,Box Kunci 1 Muka,84727.00,4.000,3.389080e+04,1,560540673980555,0
...,...,...,...,...,...,...,...,...,...
3630143,66695170713,6469551952,Blockboard Core 24K 24 x 960 x 1870 mm,6255355.00,17.234,1.078048e+07,12,303865310068555,867464278068555
3630144,66781327648,6474881364,KASUR PRINCE 180,8181818.20,4.000,3.272727e+06,12,569089169256555,0
3630145,66694924760,6460156817,FULL SET M/P FITTED JERSEY PU 200X200,6674545.33,2.000,1.334909e+06,12,560977158960555,510039378071555
3630146,66597008664,6347721868,FREIGHT FORWARDING SERVICES,8835555.00,1.000,8.835555e+05,12,568036144517555,569408433502555


# FP masukan untuk NPWP Pembeli

# FP keluaran untuk NPWP penjual

# Tidak ada NPWP Penjual = 0 

In [153]:
dfinal_obyeks_keluar = dfinal_obyek[['PPN barang','MASA_PAJAK','NPWP_PENJUAL']]
dfinal_obyeks_masukan = dfinal_obyek[['PPN barang','MASA_PAJAK','NPWP_PEMBELI']]

# FP Masukan untuk NPWP Pembeli

In [130]:
NPWP = []
FP_1=[]
FP_2=[]
FP_3=[]
FP_4=[]
FP_5=[]
FP_6=[]
FP_7=[]
FP_8=[]
FP_9=[]
FP_10=[]
FP_11=[]
FP_12=[]
for i in tqdm(range(len(dfinal_obyeks_masukan.NPWP_PEMBELI.unique()))):
    df_user = dfinal_obyeks_masukan[dfinal_obyeks_masukan.NPWP_PEMBELI==dfinal_obyeks_masukan.NPWP_PEMBELI.unique()[i]]
    
    NPWP.append(df_user.NPWP_PEMBELI.iloc[0])
    
    FP_1.append(df_user.loc[df_user['MASA_PAJAK'] == 1]['PPN barang'].sum())
    FP_2.append(df_user.loc[df_user['MASA_PAJAK'] == 2]['PPN barang'].sum())
    FP_3.append(df_user.loc[df_user['MASA_PAJAK'] == 3]['PPN barang'].sum())
    
    FP_4.append(df_user.loc[df_user['MASA_PAJAK'] == 4]['PPN barang'].sum())
    FP_5.append(df_user.loc[df_user['MASA_PAJAK'] == 5]['PPN barang'].sum())
    FP_6.append(df_user.loc[df_user['MASA_PAJAK'] == 6]['PPN barang'].sum())
    
    FP_7.append(df_user.loc[df_user['MASA_PAJAK'] == 7]['PPN barang'].sum())
    FP_8.append(df_user.loc[df_user['MASA_PAJAK'] == 8]['PPN barang'].sum())
    FP_9.append(df_user.loc[df_user['MASA_PAJAK'] == 9]['PPN barang'].sum())
    
    FP_10.append(df_user.loc[df_user['MASA_PAJAK'] == 10]['PPN barang'].sum())
    FP_11.append(df_user.loc[df_user['MASA_PAJAK'] == 11]['PPN barang'].sum())
    FP_12.append(df_user.loc[df_user['MASA_PAJAK'] == 12]['PPN barang'].sum())
    
pd_faktur_masukan =pd.DataFrame()

pd_faktur_masukan['NPWP']=NPWP

pd_faktur_masukan['FP_masukan1']=FP_1
pd_faktur_masukan['FP_masukan2']=FP_2
pd_faktur_masukan['FP_masukan3']=FP_3
pd_faktur_masukan['FP_masukan4']=FP_4
pd_faktur_masukan['FP_masukan5']=FP_5
pd_faktur_masukan['FP_masukan6']=FP_6
pd_faktur_masukan['FP_masukan7']=FP_7
pd_faktur_masukan['FP_masukan8']=FP_8
pd_faktur_masukan['FP_masukan9']=FP_9
pd_faktur_masukan['FP_masukan10']=FP_10
pd_faktur_masukan['FP_masukan11']=FP_11
pd_faktur_masukan['FP_masukan12']=FP_12

pd_faktur_masukan

100%|████████████████████████████████████████████████████████████████████████████| 13651/13651 [09:24<00:00, 24.19it/s]


,NPWP,FP_masukan1,FP_masukan2,FP_masukan3,FP_masukan4,FP_masukan5,FP_masukan6,FP_masukan7,FP_masukan8,FP_masukan9,FP_masukan10,FP_masukan11,FP_masukan12
0,0,3.068844e+15,4.173766e+14,1.554163e+14,1.562728e+14,5.946935e+14,1.209961e+14,1.181493e+14,9.069517e+13,1.228736e+14,3.000395e+14,1.266744e+14,3.009251e+14
1,560367192197555,4.192904e+12,1.921666e+11,9.985709e+11,1.342687e+12,3.003329e+11,3.088484e+11,5.160773e+11,9.835162e+12,2.503874e+11,1.909030e+11,1.817018e+11,2.645958e+11
2,562197926987555,8.392886e+11,1.490833e+12,4.320678e+11,2.748449e+11,4.436590e+11,2.819603e+11,3.161749e+11,2.729452e+11,2.268454e+11,2.838937e+11,1.686033e+11,1.293192e+11
3,765266892960555,8.416021e+08,4.522693e+08,4.571946e+08,3.747036e+08,4.369698e+08,5.337475e+08,5.737941e+08,5.679394e+08,6.010862e+08,2.832987e+09,4.783360e+09,1.640089e+09
4,365648970169555,4.370842e+11,5.216694e+11,7.466888e+11,8.435585e+11,2.553291e+11,5.844137e+11,6.373832e+11,7.340788e+11,1.438344e+11,3.474337e+11,3.941156e+11,5.822228e+10
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13646,563791695546555,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,7.245877e+09
13647,392936230159555,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.742222e+09,0.000000e+00,0.000000e+00
13648,566514811096556,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.945694e+10,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
13649,560124088186555,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.298485e+07,0.000000e+00


In [154]:
pd_faktur_masukan

,NPWP,FP_masukan1,FP_masukan2,FP_masukan3,FP_masukan4,FP_masukan5,FP_masukan6,FP_masukan7,FP_masukan8,FP_masukan9,FP_masukan10,FP_masukan11,FP_masukan12
0,0,3.068844e+15,4.173766e+14,1.554163e+14,1.562728e+14,5.946935e+14,1.209961e+14,1.181493e+14,9.069517e+13,1.228736e+14,3.000395e+14,1.266744e+14,3.009251e+14
1,560367192197555,4.192904e+12,1.921666e+11,9.985709e+11,1.342687e+12,3.003329e+11,3.088484e+11,5.160773e+11,9.835162e+12,2.503874e+11,1.909030e+11,1.817018e+11,2.645958e+11
2,562197926987555,8.392886e+11,1.490833e+12,4.320678e+11,2.748449e+11,4.436590e+11,2.819603e+11,3.161749e+11,2.729452e+11,2.268454e+11,2.838937e+11,1.686033e+11,1.293192e+11
3,765266892960555,8.416021e+08,4.522693e+08,4.571946e+08,3.747036e+08,4.369698e+08,5.337475e+08,5.737941e+08,5.679394e+08,6.010862e+08,2.832987e+09,4.783360e+09,1.640089e+09
4,365648970169555,4.370842e+11,5.216694e+11,7.466888e+11,8.435585e+11,2.553291e+11,5.844137e+11,6.373832e+11,7.340788e+11,1.438344e+11,3.474337e+11,3.941156e+11,5.822228e+10
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13646,563791695546555,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,7.245877e+09
13647,392936230159555,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.742222e+09,0.000000e+00,0.000000e+00
13648,566514811096556,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.945694e+10,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
13649,560124088186555,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.298485e+07,0.000000e+00


# FP Keluaran untuk NPWP Penjual

In [131]:
NPWP = []
FP_1=[]
FP_2=[]
FP_3=[]
FP_4=[]
FP_5=[]
FP_6=[]
FP_7=[]
FP_8=[]
FP_9=[]
FP_10=[]
FP_11=[]
FP_12=[]
for i in tqdm(range(len(dfinal_obyeks_keluar.NPWP_PENJUAL.unique()))):
    df_user = dfinal_obyeks_keluar[dfinal_obyeks_keluar.NPWP_PENJUAL==dfinal_obyeks_keluar.NPWP_PENJUAL.unique()[i]]
    
    NPWP.append(df_user.NPWP_PENJUAL.iloc[0])
    
    FP_1.append(df_user.loc[df_user['MASA_PAJAK'] == 1]['PPN barang'].sum())
    FP_2.append(df_user.loc[df_user['MASA_PAJAK'] == 2]['PPN barang'].sum())
    FP_3.append(df_user.loc[df_user['MASA_PAJAK'] == 3]['PPN barang'].sum())
    
    FP_4.append(df_user.loc[df_user['MASA_PAJAK'] == 4]['PPN barang'].sum())
    FP_5.append(df_user.loc[df_user['MASA_PAJAK'] == 5]['PPN barang'].sum())
    FP_6.append(df_user.loc[df_user['MASA_PAJAK'] == 6]['PPN barang'].sum())
    
    FP_7.append(df_user.loc[df_user['MASA_PAJAK'] == 7]['PPN barang'].sum())
    FP_8.append(df_user.loc[df_user['MASA_PAJAK'] == 8]['PPN barang'].sum())
    FP_9.append(df_user.loc[df_user['MASA_PAJAK'] == 9]['PPN barang'].sum())
    
    FP_10.append(df_user.loc[df_user['MASA_PAJAK'] == 10]['PPN barang'].sum())
    FP_11.append(df_user.loc[df_user['MASA_PAJAK'] == 11]['PPN barang'].sum())
    FP_12.append(df_user.loc[df_user['MASA_PAJAK'] == 12]['PPN barang'].sum())
    
pd_faktur_keluar =pd.DataFrame()

pd_faktur_keluar['NPWP']=NPWP

pd_faktur_keluar['FP_keluaran1']=FP_1
pd_faktur_keluar['FP_keluaran2']=FP_2
pd_faktur_keluar['FP_keluaran3']=FP_3
pd_faktur_keluar['FP_keluaran4']=FP_4
pd_faktur_keluar['FP_keluaran5']=FP_5
pd_faktur_keluar['FP_keluaran6']=FP_6
pd_faktur_keluar['FP_keluaran7']=FP_7
pd_faktur_keluar['FP_keluaran8']=FP_8
pd_faktur_keluar['FP_keluaran9']=FP_9
pd_faktur_keluar['FP_keluaran10']=FP_10
pd_faktur_keluar['FP_keluaran11']=FP_11
pd_faktur_keluar['FP_keluaran12']=FP_12

pd_faktur_keluar

100%|████████████████████████████████████████████████████████████████████████████| 15272/15272 [10:49<00:00, 23.53it/s]


,NPWP,FP_keluaran1,FP_keluaran2,FP_keluaran3,FP_keluaran4,FP_keluaran5,FP_keluaran6,FP_keluaran7,FP_keluaran8,FP_keluaran9,FP_keluaran10,FP_keluaran11,FP_keluaran12
0,562197926987555,1.924957e+11,1.305442e+11,1.825673e+11,1.263300e+11,1.459558e+11,1.518127e+11,1.639805e+11,1.125013e+11,9.392865e+10,1.368818e+11,9.386184e+10,1.138923e+11
1,560540673980555,1.783703e+10,5.108339e+10,4.165735e+10,3.596636e+10,3.530858e+10,3.347562e+10,3.842335e+10,3.436134e+10,2.730653e+10,4.771034e+10,2.531442e+10,5.458747e+10
2,568127133583555,6.167483e+10,3.460207e+10,5.022037e+10,4.912525e+10,3.227907e+10,5.360279e+10,6.551359e+10,5.590526e+10,3.169133e+10,4.400806e+10,3.035041e+10,3.068603e+10
3,561725390986555,3.811280e+11,2.634146e+11,8.451206e+11,8.405869e+11,5.934585e+11,2.220872e+11,9.559369e+12,1.614095e+12,1.560891e+12,1.263877e+12,3.566923e+13,7.790806e+12
4,575401131957555,6.734168e+11,7.035141e+11,5.206222e+10,1.271605e+11,5.266616e+11,7.959143e+11,5.808732e+10,1.885780e+10,8.631564e+09,6.724246e+11,9.265211e+09,7.041395e+11
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15267,265046529568555,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,5.555500e+03,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
15268,574405549558555,0.000000e+00,0.000000e+00,3.111110e+05,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
15269,574023811593555,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.590448e+06,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
15270,574485675563555,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,6.045454e+05,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00


In [145]:
pd_faktur_keluar.NPWP.nunique()

15272

# Join Tabel faktur keluaran dan faktur masukan berdasarkan NPWP

In [147]:
dfinal_faktur = pd_faktur_masukan.merge(pd_faktur_keluar, on="NPWP", how = 'inner')
dfinal_faktur

,NPWP,FP_masukan1,FP_masukan2,FP_masukan3,FP_masukan4,FP_masukan5,FP_masukan6,FP_masukan7,FP_masukan8,FP_masukan9,...,FP_keluaran3,FP_keluaran4,FP_keluaran5,FP_keluaran6,FP_keluaran7,FP_keluaran8,FP_keluaran9,FP_keluaran10,FP_keluaran11,FP_keluaran12
0,560367192197555,4.192904e+12,1.921666e+11,9.985709e+11,1.342687e+12,3.003329e+11,3.088484e+11,5.160773e+11,9.835162e+12,2.503874e+11,...,1.066175e+09,3.502719e+09,4.129131e+09,5.317435e+09,8.150473e+09,4.304880e+08,4.839532e+07,1.897840e+09,1.824086e+09,2.983770e+09
1,562197926987555,8.392886e+11,1.490833e+12,4.320678e+11,2.748449e+11,4.436590e+11,2.819603e+11,3.161749e+11,2.729452e+11,2.268454e+11,...,1.825673e+11,1.263300e+11,1.459558e+11,1.518127e+11,1.639805e+11,1.125013e+11,9.392865e+10,1.368818e+11,9.386184e+10,1.138923e+11
2,765266892960555,8.416021e+08,4.522693e+08,4.571946e+08,3.747036e+08,4.369698e+08,5.337475e+08,5.737941e+08,5.679394e+08,6.010862e+08,...,1.745455e+07,8.727273e+06,3.014591e+06,2.156828e+06,1.520465e+06,6.818182e+06,9.545454e+05,2.669409e+07,2.722073e+08,1.004545e+07
3,565164624502555,4.959494e+12,4.650089e+12,2.316543e+12,4.464141e+12,5.771980e+12,1.158222e+12,4.160573e+12,1.902679e+14,9.778213e+11,...,4.852217e+08,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
4,587952602257555,2.855279e+11,2.785060e+11,4.323493e+11,3.754511e+11,4.503848e+11,3.479993e+11,4.130550e+11,4.123683e+11,1.375919e+11,...,6.049986e+10,5.314448e+10,5.882152e+10,6.050520e+10,5.737608e+10,5.780703e+10,1.694025e+10,5.662173e+10,9.535973e+11,8.965202e+10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2527,866433266966555,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,7.375555e+05,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.091547e+07,0.000000e+00,3.732778e+06
2528,560977851960555,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,1.314997e+12,8.600867e+11,5.482983e+11,9.147472e+11,1.270422e+12,1.346926e+12,7.056667e+10,1.248010e+12,1.045967e+11,2.184936e+11
2529,560363272400555,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,3.796294e+08,0.000000e+00,0.000000e+00,0.000000e+00,...,1.545455e+09,6.909091e+08,1.363636e+09,4.742084e+10,0.000000e+00,0.000000e+00,0.000000e+00,3.577248e+13,5.194743e+13,9.161394e+11
2530,585737579058555,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,6.438812e+07,...,4.238863e+04,0.000000e+00,0.000000e+00,8.949977e+07,0.000000e+00,0.000000e+00,0.000000e+00,2.374167e+07,2.194027e+07,3.815891e+07


In [138]:
list_jual = pd_faktur_masukan.NPWP.unique().tolist()
list_beli = pd_faktur_keluar.NPWP.unique().tolist()

# Perhitungan faktur total per NPWP pada masa pajak tertentu

# Faktur total = Faktur masukan - Faktur Keluaran

In [139]:
for i in range(1,13):
    dfinal_faktur['FP_{}'.format(i)] = dfinal_faktur['FP_masukan{}'.format(i)] - dfinal_faktur['FP_keluaran{}'.format(i)]

In [140]:
dfinal_faktur_npwp = dfinal_faktur[['FP_1','FP_2','FP_3','FP_4','FP_5','FP_6',
              'FP_7','FP_8','FP_9','FP_10','FP_11','FP_12']]

In [141]:
dfinal_faktur_npwp['NPWP'] = dfinal_faktur.NPWP.tolist()

C:\Users\hrd\.conda\envs\py_TF2\lib\site-packages\pandas\core\frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [142]:
dfinal_faktur_npwp.to_csv('20210731_Data Faktur per NPWP per masa pajak.csv')

In [144]:
dfinal_faktur_npwp

,FP_1,FP_2,FP_3,FP_4,FP_5,FP_6,FP_7,FP_8,FP_9,FP_10,FP_11,FP_12,NPWP
0,6.467929e+11,1.360289e+12,2.495005e+11,1.485149e+11,2.977032e+11,1.301476e+11,1.521944e+11,1.604439e+11,1.329168e+11,1.470119e+11,7.474144e+10,1.542694e+10,562197926987555
1,1.921724e+12,6.836181e+11,3.637846e+12,1.861282e+11,9.103713e+11,2.880675e+12,1.614540e+12,1.087428e+12,1.563684e+12,2.662808e+12,1.642594e+12,1.584877e+11,560540673980555
2,-6.167469e+10,-3.460207e+10,-3.711834e+10,-3.696261e+10,-2.112113e+10,-5.360252e+10,-4.970846e+10,-5.230391e+10,-3.138140e+10,-3.767115e+10,-2.017405e+10,-2.419384e+10,568127133583555
3,-3.811280e+11,-2.634144e+11,-8.451206e+11,-8.405869e+11,-5.934585e+11,-2.220851e+11,-9.559368e+12,-1.594444e+12,-1.560891e+12,-1.263877e+12,-3.566923e+13,-7.790806e+12,561725390986555
4,9.366596e+12,2.939077e+12,6.061455e+12,2.116758e+13,7.450490e+12,5.746475e+12,3.431415e+12,4.151862e+12,4.373211e+12,6.238752e+12,3.920092e+12,3.230644e+12,575401131957555
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2527,0.000000e+00,0.000000e+00,0.000000e+00,3.338462e+06,0.000000e+00,2.176871e+10,0.000000e+00,0.000000e+00,3.506555e+07,1.530619e+07,1.083740e+07,2.131758e+10,568784438516555
2528,0.000000e+00,0.000000e+00,0.000000e+00,5.964444e+10,5.452445e+13,0.000000e+00,0.000000e+00,0.000000e+00,1.232894e+10,0.000000e+00,0.000000e+00,-1.217552e+12,382179112966555
2529,4.825321e+07,4.878519e+07,9.783675e+08,1.416419e+09,4.223215e+08,2.679173e+08,8.188523e+08,2.865150e+08,3.042806e+08,2.036291e+08,3.645677e+08,3.442773e+08,537889649662555
2530,0.000000e+00,0.000000e+00,1.512807e+08,0.000000e+00,4.431082e+08,4.605801e+07,1.932433e+10,1.089284e+10,3.110116e+10,0.000000e+00,0.000000e+00,-1.796265e+10,570003130580555


# Load Data hasil olahan Faktur Pajak

In [215]:
# FP masukan
pd_faktur_masukan
#FP kluaran
pd_faktur_keluar

dfinal_faktur = pd.read_csv('20210731_Data Faktur per NPWP per masa pajak.csv')

In [216]:
dfinal_faktur=dfinal_faktur[['NPWP','FP_1','FP_2','FP_3','FP_4','FP_5'
                            ,'FP_6','FP_7','FP_8','FP_9','FP_10','FP_11','FP_12']]
dfinal_faktur.head()

,NPWP,FP_1,FP_2,FP_3,FP_4,FP_5,FP_6,FP_7,FP_8,FP_9,FP_10,FP_11,FP_12
0,562197926987555,6.467929e+11,1.360289e+12,2.495005e+11,1.485149e+11,2.977032e+11,1.301476e+11,1.521944e+11,1.604439e+11,1.329168e+11,1.470119e+11,7.474144e+10,1.542694e+10
1,560540673980555,1.921724e+12,6.836181e+11,3.637846e+12,1.861282e+11,9.103713e+11,2.880675e+12,1.614540e+12,1.087428e+12,1.563684e+12,2.662808e+12,1.642594e+12,1.584877e+11
2,568127133583555,-6.167469e+10,-3.460207e+10,-3.711834e+10,-3.696261e+10,-2.112113e+10,-5.360252e+10,-4.970846e+10,-5.230391e+10,-3.138140e+10,-3.767115e+10,-2.017405e+10,-2.419384e+10
3,561725390986555,-3.811280e+11,-2.634144e+11,-8.451206e+11,-8.405869e+11,-5.934585e+11,-2.220851e+11,-9.559368e+12,-1.594444e+12,-1.560891e+12,-1.263877e+12,-3.566923e+13,-7.790806e+12
4,575401131957555,9.366596e+12,2.939077e+12,6.061455e+12,2.116758e+13,7.450490e+12,5.746475e+12,3.431415e+12,4.151862e+12,4.373211e+12,6.238752e+12,3.920092e+12,3.230644e+12


In [217]:
list_faktur_masuk = pd_faktur_masukan.NPWP.unique().tolist()
list_faktur_keluar = pd_faktur_keluar.NPWP.unique().tolist()
list_final_faktur = dfinal_faktur.NPWP.unique().tolist()

len(list(set(list_faktur_masuk) & set(list_faktur_keluar)))

len(list_faktur_keluar)

15272

In [218]:
list_faktur_keluar_outer = list(set(list_faktur_keluar) - set(list_final_faktur))
list_faktur_masuk_outer = list(set(list_faktur_masuk) - set(list_final_faktur))

In [219]:
#Olah faktur masukan extended
#faktur yang tidak memiliki NPWP pembeli / NPWP penjual
faktur_masukan_extra = pd_faktur_masukan[pd_faktur_masukan['NPWP'].isin(list_faktur_masuk_outer)]
faktur_keluaran_extra = pd_faktur_keluar[pd_faktur_keluar['NPWP'].isin(list_faktur_keluar_outer)]


In [220]:
#Faktur Pajak Masukan Extra
for i in range(1,13):
    faktur_masukan_extra['FP_{}'.format(i)] = faktur_masukan_extra['FP_masukan{}'.format(i)] - 0

faktur_masukan_extra.head()

faktur_masukan_extra = faktur_masukan_extra.drop(columns=['FP_masukan1','FP_masukan2','FP_masukan3','FP_masukan4'
                                                         ,'FP_masukan5','FP_masukan6','FP_masukan7','FP_masukan8'
                                                         ,'FP_masukan9','FP_masukan10','FP_masukan11','FP_masukan12'])

faktur_masukan_extra.head()

C:\Users\hrd\.conda\envs\py_TF2\lib\site-packages\pandas\core\frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


,NPWP,FP_1,FP_2,FP_3,FP_4,FP_5,FP_6,FP_7,FP_8,FP_9,FP_10,FP_11,FP_12
0,0,3.068844e+15,4.173766e+14,1.554163e+14,1.562728e+14,5.946935e+14,1.209961e+14,1.181493e+14,9.069517e+13,1.228736e+14,3.000395e+14,1.266744e+14,3.009251e+14
4,365648970169555,4.370842e+11,5.216694e+11,7.466888e+11,8.435585e+11,2.553291e+11,5.844137e+11,6.373832e+11,7.340788e+11,1.438344e+11,3.474337e+11,3.941156e+11,5.822228e+10
5,110188524518555,6.686230e+08,1.052579e+09,0.000000e+00,0.000000e+00,1.029290e+09,1.046735e+09,7.206111e+06,0.000000e+00,1.676836e+07,0.000000e+00,1.552501e+08,5.425600e+06
7,586449816578555,2.910618e+07,4.243536e+07,7.132402e+06,3.113693e+07,2.584751e+07,1.148563e+07,3.370239e+08,1.903650e+07,1.613435e+07,6.875854e+07,1.360423e+07,2.856317e+07
9,765041871906555,8.525688e+06,2.137472e+08,2.911379e+06,1.016701e+07,2.033620e+08,4.234452e+06,2.331488e+06,5.421712e+06,1.567538e+07,1.864093e+06,2.052540e+07,6.991138e+05


In [221]:
#Faktur Pajak Masukan Extra
for i in range(1,13):
    faktur_keluaran_extra['FP_{}'.format(i)] = faktur_keluaran_extra['FP_keluaran{}'.format(i)] - 0

faktur_keluaran_extra.head()

faktur_keluaran_extra = faktur_keluaran_extra.drop(columns=['FP_keluaran1','FP_keluaran2','FP_keluaran3','FP_keluaran4'
                                                         ,'FP_keluaran5','FP_keluaran6','FP_keluaran7','FP_keluaran8'
                                                         ,'FP_keluaran9','FP_keluaran10','FP_keluaran11','FP_keluaran12'])

faktur_keluaran_extra.head()

C:\Users\hrd\.conda\envs\py_TF2\lib\site-packages\pandas\core\frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


,NPWP,FP_1,FP_2,FP_3,FP_4,FP_5,FP_6,FP_7,FP_8,FP_9,FP_10,FP_11,FP_12
10,577417790584555,1.702395e+11,1.766347e+10,1.797830e+10,2.050135e+10,9.672570e+09,7.716133e+10,1.443531e+10,1.322175e+11,4.229373e+10,1.419775e+10,2.930758e+10,5.019886e+10
12,570016332963555,2.060985e+07,8.727280e+04,1.267269e+08,0.000000e+00,0.000000e+00,1.429311e+08,1.839538e+08,2.513455e+08,0.000000e+00,2.628410e+07,2.237155e+07,4.405162e+06
14,860384042906555,2.487568e+12,1.231394e+12,1.208170e+12,1.994687e+12,6.945421e+11,1.236690e+12,1.224490e+12,2.459458e+12,6.058703e+11,2.080369e+12,1.385456e+12,2.778157e+12
16,563226791571555,1.305439e+09,1.174170e+09,1.397789e+09,1.329529e+09,1.092847e+09,8.580404e+08,1.176813e+09,6.855318e+08,9.355644e+08,8.417050e+08,1.147986e+09,8.077827e+08
18,563792717507555,2.478925e+11,5.308189e+11,1.876075e+11,2.619287e+11,2.848637e+11,3.638719e+11,3.518953e+12,2.672112e+11,1.026582e+11,3.215495e+11,2.893014e+11,3.568527e+11


In [222]:
dfinal_faktur = dfinal_faktur.append(faktur_masukan_extra)

In [223]:
dfinal_faktur = dfinal_faktur.append(faktur_keluaran_extra)
dfinal_faktur = dfinal_faktur[dfinal_faktur.NPWP != 0]

In [226]:
dfinal_faktur.to_csv('Data Final faktur setiap masa pajak seluruh NPWP.csv')

# Olah data faktur Final all NPWP

# Fitur yang perlu ditambah :

## 1.Apakah NPWP masuk kedalam list impor
## 2.Apakah NPWP masuk kedalam list export
## 3.Apakah NPWP masuk kedalam list data wajib pajak
## 4.score LDA topik 1-5 untuk barang di tiap NPWP

## 5.cari selisih CIF & FOBR untuk tiap masa pajak untuk daftar NPWP yang termasuk

## 6.Jumlah unik NPWP lain yang bertransaksi dengan NPWP tersebut
## 7.Nilai maksimum jumlah transaksi dengan NPWP yang sama

In [366]:
dfaktur_final = pd.read_csv('Data Final faktur setiap masa pajak seluruh NPWP.csv')

In [369]:
list(set(dfaktur_final.NPWP.unique().tolist()) & set(data_wajib_pajak.NPWP.unique()))

[96180364951555,
 587587806060555,
 450820647190555,
 561623874453555,
 457352535957555,
 586486356074555,
 560114442559555,
 575363663167555,
 567389389971555,
 860729430163555,
 596864939581555,
 567238626877555,
 572135437074555,
 577448290066555,
 587810743058555,
 568077082599555,
 277825757159555,
 862796411068555,
 512031793553555,
 395694701969555,
 569967521169555,
 860098661990555,
 585039461179555,
 569408722960555,
 569357780197555,
 206205770170555,
 568250174906555,
 212706185060555,
 381466635108555,
 595543296057555,
 562932710670555,
 860223935971555,
 570663612088555,
 587468419170555,
 595977082978555,
 566980913676555,
 568267352588555,
 565146136502555,
 563792792502555,
 562728856502555,
 375186688950555,
 575035014070555,
 578618644406555,
 564673272971555,
 575363671179555,
 577520120160555,
 561012993589555,
 566853393973555,
 205937340981555,
 212971629109555,
 766939952906555,
 575097139167555,
 562786466996555,
 512372138164555,
 564285521076555,
 8690033461

In [370]:
list_data_importir   = data_importir.NPWP_IMPORTIR.unique().tolist()
list_data_eksportir  = data_eksportir.NPWP_EKSPORTIR.unique().tolist()
list_data_wajib_pajak=data_wajib_pajak.NPWP.unique().tolist()

In [371]:
len(list_data_eksportir)

665

# Menambah Beberapa Fitur untuk Tabel Data Faktur NPWP

In [372]:
list_data_importir_set =set(list_data_importir) 
list_data_eksportir_set=set(list_data_eksportir)
list_data_wajib_pajak_set=set(list_data_wajib_pajak)

is_importir=[]
is_eksportir=[]
is_wajib_pajak=[]
#Importir
for i in range(dfaktur_final.shape[0]):
    if dfaktur_final.NPWP.iloc[i] in list_data_importir_set :
        is_importir.append(1)
    else:
        is_importir.append(0)
        
#Eksportir
for j in range(dfaktur_final.shape[0]):
    if dfaktur_final.NPWP.iloc[j] in list_data_eksportir_set :
        is_eksportir.append(1)
    else:
        is_eksportir.append(0)
        
#Wajib Pajak
for k in range(dfaktur_final.shape[0]):
    if dfaktur_final.NPWP.iloc[k] in list_data_wajib_pajak_set :
        is_wajib_pajak.append(1)
    else:
        is_wajib_pajak.append(0)
        
dfaktur_final['is_importir'] = is_importir
dfaktur_final['is_eksportir']= is_eksportir
dfaktur_final['is_wajib_pajak']= is_wajib_pajak

dfaktur_final.head()

,Unnamed: 0,Unnamed: 0.1,NPWP,FP_1,FP_2,FP_3,FP_4,FP_5,FP_6,FP_7,FP_8,FP_9,FP_10,FP_11,FP_12,is_importir,is_eksportir,is_wajib_pajak
0,0,0,562197926987555,6.467929e+11,1.360289e+12,2.495005e+11,1.485149e+11,2.977032e+11,1.301476e+11,1.521944e+11,1.604439e+11,1.329168e+11,1.470119e+11,7.474144e+10,1.542694e+10,0,0,1
1,1,1,560540673980555,1.921724e+12,6.836181e+11,3.637846e+12,1.861282e+11,9.103713e+11,2.880675e+12,1.614540e+12,1.087428e+12,1.563684e+12,2.662808e+12,1.642594e+12,1.584877e+11,0,0,1
2,2,2,568127133583555,-6.167469e+10,-3.460207e+10,-3.711834e+10,-3.696261e+10,-2.112113e+10,-5.360252e+10,-4.970846e+10,-5.230391e+10,-3.138140e+10,-3.767115e+10,-2.017405e+10,-2.419384e+10,0,0,0
3,3,3,561725390986555,-3.811280e+11,-2.634144e+11,-8.451206e+11,-8.405869e+11,-5.934585e+11,-2.220851e+11,-9.559368e+12,-1.594444e+12,-1.560891e+12,-1.263877e+12,-3.566923e+13,-7.790806e+12,0,0,0
4,4,4,575401131957555,9.366596e+12,2.939077e+12,6.061455e+12,2.116758e+13,7.450490e+12,5.746475e+12,3.431415e+12,4.151862e+12,4.373211e+12,6.238752e+12,3.920092e+12,3.230644e+12,1,0,1


In [373]:
dfaktur_final.is_wajib_pajak.value_counts()

0    24882
1     1508
Name: is_wajib_pajak, dtype: int64

In [374]:
dfaktur_final.to_csv('Data Final faktur setiap masa pajak seluruh NPWP.csv')

In [244]:
dfinal_obyek

,ID_OBJEK_FAKTUR,ID_FAKTUR,NAMA_BARANG,DPP,JUMLAH_BARANG,PPN barang,MASA_PAJAK,NPWP_PENJUAL,NPWP_PEMBELI
0,66806169458,6406426318,BIG 102 BC,81555.00,10.000,8.155500e+04,1,562197926987555,0
1,3046030121,6984147727,Bearing Tan 6204 zz,604095.00,30.000,1.812285e+06,1,560540673980555,0
2,3046030129,6984147727,Gantungan Gembok,70205.00,2.000,1.404100e+04,1,560540673980555,0
3,3046030127,6984147727,Handle Combined,24279.00,4.000,9.711600e+03,1,560540673980555,0
4,3046030125,6984147727,Box Kunci 1 Muka,84727.00,4.000,3.389080e+04,1,560540673980555,0
...,...,...,...,...,...,...,...,...,...
3630143,66695170713,6469551952,Blockboard Core 24K 24 x 960 x 1870 mm,6255355.00,17.234,1.078048e+07,12,303865310068555,867464278068555
3630144,66781327648,6474881364,KASUR PRINCE 180,8181818.20,4.000,3.272727e+06,12,569089169256555,0
3630145,66694924760,6460156817,FULL SET M/P FITTED JERSEY PU 200X200,6674545.33,2.000,1.334909e+06,12,560977158960555,510039378071555
3630146,66597008664,6347721868,FREIGHT FORWARDING SERVICES,8835555.00,1.000,8.835555e+05,12,568036144517555,569408433502555


# Tambah fitur LDA Score untuk tiap barang yg ada pada tiap NPWP

In [250]:
#NPWP Penjual 
npwp_penjual = dfinal_obyek.groupby(['NPWP_PENJUAL','MASA_PAJAK'], as_index=False).agg({"PPN barang": "sum"})
list_barang = []
for i in tqdm(range(npwp_penjual.shape[0])):
    data_barang = dfinal_obyek.loc[(dfinal_obyek['NPWP_PENJUAL'] == npwp_penjual['NPWP_PENJUAL'].iloc[i]) & (dfinal_obyek['MASA_PAJAK'] == npwp_penjual['MASA_PAJAK'].iloc[i])]
    list_barang.append(data_barang.NAMA_BARANG.tolist())
    
len(list_barang)

100%|████████████████████████████████████████████████████████████████████████████| 92067/92067 [18:28<00:00, 83.04it/s]


92067

In [251]:
npwp_penjual['NAMA_BARANG']=list_barang

In [252]:
npwp_penjual

,NPWP_PENJUAL,MASA_PAJAK,PPN barang,NAMA_BARANG
0,30571561990555,4,3.661304e+11,"[BENANG DN MERAH 20/3. ETK, BENANG DN KUNING 2..."
1,31022230063555,3,2.990731e+07,"[CANAL 75 X 0.75 X 6.0 IBI TRUSS, RENG I - 100..."
2,31022230063555,6,3.249440e+08,"[CANAL 75 X 0.60 X 6.0 IBI TRUSS, HOLLOW GALVA..."
3,31022230063555,7,8.185449e+09,[RENG I - 1 E LIPAT]
4,36378431059555,3,1.533298e+08,"[CIRCLE SAW ONCI 300X3.2/2.2X70XZ24+V4, SKS EX..."
...,...,...,...,...
92062,999281756452555,6,2.810455e+07,"[Plakat Logo UNUD : I sebanyak 116 biji, Plaka..."
92063,999281756452555,8,1.351954e+07,"[Pembelian Plakat Logo UNUD 67 buah, Pembelian..."
92064,999281756452555,10,7.592272e+06,[Plakat Logo UNUD]
92065,999281756452555,11,7.727285e+05,[Pembuatan papan nama informasi PPID berbahan ...


In [253]:
#NPWP Pembeli
npwp_pembeli = dfinal_obyek.groupby(['NPWP_PEMBELI','MASA_PAJAK'], as_index=False).agg({"PPN barang": "sum"})
list_barang1 = []
for i in tqdm(range(npwp_pembeli.shape[0])):
    data_barang = dfinal_obyek.loc[(dfinal_obyek['NPWP_PEMBELI'] == npwp_pembeli['NPWP_PEMBELI'].iloc[i]) & (dfinal_obyek['MASA_PAJAK'] == npwp_pembeli['MASA_PAJAK'].iloc[i])]
    list_barang1.append(data_barang.NAMA_BARANG.tolist())
    
len(list_barang1)

100%|████████████████████████████████████████████████████████████████████████████| 71648/71648 [14:13<00:00, 83.92it/s]


71648

In [254]:
npwp_pembeli['NAMA_BARANG']=list_barang1

In [277]:
npwp_pembeli = npwp_pembeli[npwp_pembeli.NPWP_PEMBELI != 0]
npwp_pembeli = npwp_pembeli.rename(columns={"NPWP_PEMBELI": "NPWP"})


In [267]:
npwp_pembeli

,NPWP,MASA_PAJAK,PPN barang,NAMA_BARANG
12,505492060555,1,2.454489e+09,"[LOG BULAT, Kayu Bulat Kel. Rimba Campur Grade..."
13,505492060555,2,3.219620e+10,"[Kayu Bulat Kel. Rimba Campur Grade A, Kayu Bu..."
14,505492060555,3,4.086745e+09,[Bengkirai/Super/STMkering/1.9990 1.9 cm x 9 c...
15,505492060555,4,4.763080e+09,"[Kayu Bulat Kel. Meranti (Grade C), Kayu bulat..."
16,505492060555,5,2.573668e+09,"[Kayu bulat Kel. Rimba Campur (Grade A), Kayu ..."
...,...,...,...,...
71643,998344033062555,12,9.120555e+05,[RESIDU GERGAJIAN]
71644,999627390276555,2,6.197273e+06,[Uang Muka Penjualan 4 Genset Yanmar Type: 6GL...
71645,999627390276555,6,6.352556e+07,[Penjualan - 4 Genset Yanmar Type : 6 GL-ET Ye...
71646,999803957198555,3,1.363022e+06,"[PINTU MD ALUR FORTIS 5P WHISPER WHITE 720, KU..."


In [265]:
npwp_penjual = npwp_penjual.rename(columns={"NPWP_PENJUAL": "NPWP"})

In [268]:
npwp_penjual

,NPWP,MASA_PAJAK,PPN barang,NAMA_BARANG
0,30571561990555,4,3.661304e+11,"[BENANG DN MERAH 20/3. ETK, BENANG DN KUNING 2..."
1,31022230063555,3,2.990731e+07,"[CANAL 75 X 0.75 X 6.0 IBI TRUSS, RENG I - 100..."
2,31022230063555,6,3.249440e+08,"[CANAL 75 X 0.60 X 6.0 IBI TRUSS, HOLLOW GALVA..."
3,31022230063555,7,8.185449e+09,[RENG I - 1 E LIPAT]
4,36378431059555,3,1.533298e+08,"[CIRCLE SAW ONCI 300X3.2/2.2X70XZ24+V4, SKS EX..."
...,...,...,...,...
92062,999281756452555,6,2.810455e+07,"[Plakat Logo UNUD : I sebanyak 116 biji, Plaka..."
92063,999281756452555,8,1.351954e+07,"[Pembelian Plakat Logo UNUD 67 buah, Pembelian..."
92064,999281756452555,10,7.592272e+06,[Plakat Logo UNUD]
92065,999281756452555,11,7.727285e+05,[Pembuatan papan nama informasi PPID berbahan ...


In [269]:
npwp_pembeli = npwp_pembeli.append(npwp_penjual)
npwp_pembeli

,NPWP,MASA_PAJAK,PPN barang,NAMA_BARANG
12,505492060555,1,2.454489e+09,"[LOG BULAT, Kayu Bulat Kel. Rimba Campur Grade..."
13,505492060555,2,3.219620e+10,"[Kayu Bulat Kel. Rimba Campur Grade A, Kayu Bu..."
14,505492060555,3,4.086745e+09,[Bengkirai/Super/STMkering/1.9990 1.9 cm x 9 c...
15,505492060555,4,4.763080e+09,"[Kayu Bulat Kel. Meranti (Grade C), Kayu bulat..."
16,505492060555,5,2.573668e+09,"[Kayu bulat Kel. Rimba Campur (Grade A), Kayu ..."
...,...,...,...,...
92062,999281756452555,6,2.810455e+07,"[Plakat Logo UNUD : I sebanyak 116 biji, Plaka..."
92063,999281756452555,8,1.351954e+07,"[Pembelian Plakat Logo UNUD 67 buah, Pembelian..."
92064,999281756452555,10,7.592272e+06,[Plakat Logo UNUD]
92065,999281756452555,11,7.727285e+05,[Pembuatan papan nama informasi PPID berbahan ...


In [270]:
npwp_pembeli.to_csv('Data Nama barang tiap masa pajak tiap NPWP.csv')

In [291]:
#'Data Nama barang tiap masa pajak tiap NPWP.csv'
#'Data Final faktur setiap masa pajak seluruh NPWP.csv'

In [300]:
npwp_barang = pd.read_csv('Data Nama barang tiap masa pajak tiap NPWP.csv')
npwp_barang = npwp_barang.drop(columns='PPN barang')

In [323]:
npwp_barang["NAMA_BARANG"].iloc[99999][2:-2]

"Foam Polystrene 5MM X 100Cm X 200Cm', 'Dinding 3D WaveBoard MDF 15 m/m SMC054 V', 'Awning Sunbrella 4626 Navy', '25 Gram NW (350pcs)', '25 Gram NW (350pcs)', '5 Gram Nw (1800pcs)"

In [324]:
#ambil value inside string nama barang
nama_barang = []
for i in tqdm(range(npwp_barang.shape[0])):
    listToStr = npwp_barang.NAMA_BARANG.iloc[i][2:-2]
    nama_barang.append(listToStr)
    
npwp_barang['NAMA_BARANG']=nama_barang

100%|███████████████████████████████████████████████████████████████████████| 163703/163703 [00:01<00:00, 96900.52it/s]


In [359]:
npwp_barang.NPWP.nunique()

26390

# Aggregasi NPWP dan nama barang tiap masa pajak

In [360]:
NPWP = []
NAMA_BARANG_1=[]
NAMA_BARANG_2=[]
NAMA_BARANG_3=[]
NAMA_BARANG_4=[]
NAMA_BARANG_5=[]
NAMA_BARANG_6=[]
NAMA_BARANG_7=[]
NAMA_BARANG_8=[]
NAMA_BARANG_9=[]
NAMA_BARANG_10=[]
NAMA_BARANG_11=[]
NAMA_BARANG_12=[]

for i in tqdm(range(len(npwp_barang.NPWP.unique()))):
    df_user = npwp_barang[npwp_barang.NPWP==npwp_barang.NPWP.unique()[i]]  
    
    NPWP.append(df_user.NPWP.iloc[0])
    
    NAMA_BARANG_1.append(df_user.loc[df_user['MASA_PAJAK'] == 1]['NAMA_BARANG'].sum())
    NAMA_BARANG_2.append(df_user.loc[df_user['MASA_PAJAK'] == 2]['NAMA_BARANG'].sum())
    NAMA_BARANG_3.append(df_user.loc[df_user['MASA_PAJAK'] == 3]['NAMA_BARANG'].sum())
    NAMA_BARANG_4.append(df_user.loc[df_user['MASA_PAJAK'] == 4]['NAMA_BARANG'].sum())
    NAMA_BARANG_5.append(df_user.loc[df_user['MASA_PAJAK'] == 5]['NAMA_BARANG'].sum())
    NAMA_BARANG_6.append(df_user.loc[df_user['MASA_PAJAK'] == 6]['NAMA_BARANG'].sum())
    NAMA_BARANG_7.append(df_user.loc[df_user['MASA_PAJAK'] == 7]['NAMA_BARANG'].sum())
    NAMA_BARANG_8.append(df_user.loc[df_user['MASA_PAJAK'] == 8]['NAMA_BARANG'].sum())
    NAMA_BARANG_9.append(df_user.loc[df_user['MASA_PAJAK'] == 9]['NAMA_BARANG'].sum())
    NAMA_BARANG_10.append(df_user.loc[df_user['MASA_PAJAK'] == 10]['NAMA_BARANG'].sum())
    NAMA_BARANG_11.append(df_user.loc[df_user['MASA_PAJAK'] == 11]['NAMA_BARANG'].sum())
    NAMA_BARANG_12.append(df_user.loc[df_user['MASA_PAJAK'] == 12]['NAMA_BARANG'].sum())
    
pd_npwp_barang = pd.DataFrame()
pd_npwp_barang['NPWP'] = NPWP
pd_npwp_barang['NAMA_BARANG_1'] = NAMA_BARANG_1
pd_npwp_barang['NAMA_BARANG_2'] = NAMA_BARANG_2
pd_npwp_barang['NAMA_BARANG_3'] = NAMA_BARANG_3
pd_npwp_barang['NAMA_BARANG_4'] = NAMA_BARANG_4
pd_npwp_barang['NAMA_BARANG_5'] = NAMA_BARANG_5
pd_npwp_barang['NAMA_BARANG_6'] = NAMA_BARANG_6
pd_npwp_barang['NAMA_BARANG_7'] = NAMA_BARANG_7
pd_npwp_barang['NAMA_BARANG_8'] = NAMA_BARANG_8
pd_npwp_barang['NAMA_BARANG_9'] = NAMA_BARANG_9
pd_npwp_barang['NAMA_BARANG_10'] = NAMA_BARANG_10
pd_npwp_barang['NAMA_BARANG_11'] = NAMA_BARANG_11
pd_npwp_barang['NAMA_BARANG_12'] = NAMA_BARANG_12

pd_npwp_barang

100%|███████████████████████████████████████████████████████████████████████████| 26390/26390 [03:35<00:00, 122.69it/s]


,NPWP,NAMA_BARANG_1,NAMA_BARANG_2,NAMA_BARANG_3,NAMA_BARANG_4,NAMA_BARANG_5,NAMA_BARANG_6,NAMA_BARANG_7,NAMA_BARANG_8,NAMA_BARANG_9,NAMA_BARANG_10,NAMA_BARANG_11,NAMA_BARANG_12
0,505492060555,"LOG BULAT', 'Kayu Bulat Kel. Rimba Campur Grad...","Kayu Bulat Kel. Rimba Campur Grade A', 'Kayu B...",Bengkirai/Super/STMkering/1.9990 1.9 cm x 9 cm...,"Kayu Bulat Kel. Meranti (Grade C)', 'Kayu bula...","Kayu bulat Kel. Rimba Campur (Grade A)', 'Kayu...","Kayu Bulat Kel. Meranti Grade C', 'Kayu Bulat ...","Kayu Bulat Kel. Meranti Grade C', 'PENJUALAN S...","Kayu Bulat Kel. Rimba Campur Grade A', 'Kayu B...","Kayu Bulat Kel. Meranti (Grade C)', 'Kayu bula...","KAYU BULAT MERANTI', 'Bengkirai/Super/STMkerin...","KAYU BULAT RIMBA CAMPURAN', 'KAYU BULAT MERANT...","KAYU BULAT KEL.MERANTI 28,92 M3', 'KAYU GERGAJ..."
1,513262052555,"Veneer Jati 0,6 All Grade Uk : 0,6 x 9~25 x 14...",0,0,0,0,0,0,0,0,0,0,0
2,2100122108555,"ARTIS BRUSH 6', 'ARTIS BRUSH ...",0,"KUAS ARTIST BRUSH 12 IN', 'KUAS ARTIST BRUSH 0...","ETERNA 633 VERNISH 4 INCH', 'ETERNA 633 VERNIS...","ETERNA 633 VERNISH 1,5 INCH', 'ETERNA 633 VERN...","KUAS E.633 2,5""', 'KUAS E.633 3/4""',...",0,"ETERNA 633 VERNISH 2 INCH', 'KUAS ARTIST BRUSH...",0,0,"KUAS ARTIST BRUSH 03 IN', 'ETERNA 633 VERNISH ...","ETERNA 633 VERNISH 3 INCH', 'ETERNA 633 VERNIS..."
3,2130386761555,0,0,0,0,0,0,"FULL SET KS FL SMILE KIDS 120,BOLSTER FLORENCE...","FULL SET KS SR M16 PERFECTION (ET) 200,PILLOW ...",0,0,0,0
4,2825102452555,0,0,0,0,0,0,0,"Portable Massage bed', 'Portable Massage Bed",0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
26385,991830669966556,"Busa Kulit Tipis', 'Busa BS/Super 200x100x30 C...","Busa Kulit Tipis', 'Busa Kulit Tipis', 'Busa S...","Busa Kulit Tipis', 'Busa Kulit Tipis', 'Busa K...","Busa Kulit Tipis', 'Busa Kulit Tipis', 'Busa K...","Busa Kulit Tipis', 'Busa Kulit Tipis', 'Busa K...","Busa Kulit Tipis', 'Busa Ancuran', 'Busa Kulit...","Busa Kulit Tipis', 'Busa Kulit Tipis', 'Busa K...","Busa Kulit Tipis', 'Busa Kulit Tipis', 'Busa K...",Busa Kulit Tipis,"Busa Kulit Tipis', 'Busa Kulit Tipis', 'Busa K...","Busa Kulit Tipis', 'Busa Kulit Tipis', 'Busa K...",Busa Kulit Tipis
26386,992307921068555,LOG ALBASIA 16-30 UP (M3),LOG ALBASIA 16-30 UP (M3),LOG ALBASIA 16-30 UP (M3),LOG ALBASIA 16-30 UP (M3),LOG ALBASIA 16-30 UP (M3),LOG ALBASIA 16-30 UP (M3),LOG ALBASIA 16-30 UP (M3),LOG ALBASIA 16-30 UP (M3),LOG ALBASIA 16-30 UP (M3),LOG ALBASIA 16-30 UP (M3),LOG ALBASIA 16-30 UP (M3),LOG ALBASIA 16-30 UP (M3)
26387,994306253068555,0,0,"Usuk Kayu Keras Uk. 4 cm x 4 cm x 3 m', 'Palet...",Palet Tutup Kaki Tiga Besar Uk. 120 x 110 x 14...,"Kayu Persegi Uk. 5 x 5x 3 m', 'Pallet Kaki Tig...",0,Pallet Kaki Tiga Besar Tanpa Tutup Uk. 120 x 1...,Palet Tutup Kaki Tiga Besar Uk. 120 x 110 x 14...,0,Palet Tutup Kaki Tiga Besar Uk. 120 x 110 x 14...,Palet Tutup Kaki Tiga Besar Uk. 120 x 110 x 14...,Palet Tutup Kaki Tiga Besar Uk. 120 x 110 x 14...
26388,998947805062555,0,0,0,0,BIAYA BELANJA BARANG (PERLENGKAPAN) DALAM RANG...,0,0,0,0,0,0,Biaya belanja barang (perlengkapan) dalam rang...


In [361]:
pd_npwp_barang.to_csv('Data Barang tiap masa pajak tiap NPWP ready for LDA.csv')

In [362]:
pd_npwp_barang.NPWP.nunique()

26390

# Aggregasi Data yang akan dipakai Modelling

In [403]:
dfinal_faktur = pd.read_csv('Data Final faktur setiap masa pajak seluruh NPWP.csv')
dfinal_scoring= pd.read_csv('Data_hasil_scoring_LDA_per_NPWP_max_value.csv') #Hasil LDA Scoring 
dfinal_faktur_riziq = pd.read_csv('agregasi_profile_NPWP.csv')  # Tambahan beberapa fitur
                                                                # Script ada di Riziq

In [404]:
dfinal_faktur.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,NPWP,FP_1,FP_2,FP_3,FP_4,FP_5,FP_6,FP_7,FP_8,FP_9,FP_10,FP_11,FP_12,is_importir,is_eksportir,is_wajib_pajak
0,0,0,0,562197926987555,6.467929e+11,1.360289e+12,2.495005e+11,1.485149e+11,2.977032e+11,1.301476e+11,1.521944e+11,1.604439e+11,1.329168e+11,1.470119e+11,7.474144e+10,1.542694e+10,0,0,1
1,1,1,1,560540673980555,1.921724e+12,6.836181e+11,3.637846e+12,1.861282e+11,9.103713e+11,2.880675e+12,1.614540e+12,1.087428e+12,1.563684e+12,2.662808e+12,1.642594e+12,1.584877e+11,0,0,1
2,2,2,2,568127133583555,-6.167469e+10,-3.460207e+10,-3.711834e+10,-3.696261e+10,-2.112113e+10,-5.360252e+10,-4.970846e+10,-5.230391e+10,-3.138140e+10,-3.767115e+10,-2.017405e+10,-2.419384e+10,0,0,0
3,3,3,3,561725390986555,-3.811280e+11,-2.634144e+11,-8.451206e+11,-8.405869e+11,-5.934585e+11,-2.220851e+11,-9.559368e+12,-1.594444e+12,-1.560891e+12,-1.263877e+12,-3.566923e+13,-7.790806e+12,0,0,0
4,4,4,4,575401131957555,9.366596e+12,2.939077e+12,6.061455e+12,2.116758e+13,7.450490e+12,5.746475e+12,3.431415e+12,4.151862e+12,4.373211e+12,6.238752e+12,3.920092e+12,3.230644e+12,1,0,1


In [407]:
dfinal_scoring.head()
dfinal_scoring = dfinal_scoring.drop(columns=['Unnamed: 0'])
dfinal_scoring

,NPWP,score_1,score_2,score_3,score_4,score_5,score_6,score_7,score_8,score_9,score_10,score_11,score_12
0,505492060555,0.547372,0.547842,0.556887,0.564858,0.418798,0.560456,0.472518,0.601539,0.510728,0.793748,0.355546,0.537923
1,513262052555,0.307239,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,2100122108555,0.910385,0.000000,0.926442,0.733111,0.606320,0.799398,0.000000,0.866057,0.000000,0.000000,0.865407,0.884864
3,2130386761555,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.535478,0.866552,0.000000,0.000000,0.000000,0.000000
4,2825102452555,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
26385,991830669966556,0.613422,0.743082,0.866637,0.799956,0.839964,0.899978,0.799956,0.839964,0.599906,0.866637,0.866637,0.599906
26386,992307921068555,0.599759,0.599767,0.599760,0.599759,0.599758,0.599756,0.599767,0.599757,0.599758,0.599761,0.599757,0.599761
26387,994306253068555,0.000000,0.000000,0.842833,0.927241,0.538963,0.000000,0.741266,0.820110,0.000000,0.860113,0.911072,0.836487
26388,998947805062555,0.000000,0.000000,0.000000,0.000000,0.630459,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.468643


In [410]:
'''
dfinal_faktur_riziq=dfinal_faktur_riziq[['NPWP','is_NPWP_pembeli_same_NPWP_penjual','Nunique_NPWP_Pembeli',
                                        'Nunique_NPWP_Penjual','max_jmlh_barang_diekspor_dlm_sehari','max_jmlh_barang_diimpor_dlm_sehari',
                                        'max_jumlah_faktur_ke_1_buyer','max_jumlah_faktur_ke_1_seller']]
'''
dfinal_faktur_riziq.head()

,NPWP,FP_1,FP_2,FP_3,FP_4,FP_5,FP_6,FP_7,FP_8,FP_9,...,is_importir,is_eksportir,is_wajib_pajak,is_NPWP_pembeli_same_NPWP_penjual,Nunique_NPWP_Pembeli,Nunique_NPWP_Penjual,max_jmlh_barang_diekspor_dlm_sehari,max_jmlh_barang_diimpor_dlm_sehari,max_jumlah_faktur_ke_1_buyer,max_jumlah_faktur_ke_1_seller
0,562197926987555,6.467929e+11,1.360289e+12,2.495005e+11,1.485149e+11,2.977032e+11,1.301476e+11,1.521944e+11,1.604439e+11,1.329168e+11,...,0,0,1,0,43.0,28.0,0.0,0.0,11468.0,3352.0
1,560540673980555,1.921724e+12,6.836181e+11,3.637846e+12,1.861282e+11,9.103713e+11,2.880675e+12,1.614540e+12,1.087428e+12,1.563684e+12,...,0,0,1,0,69.0,80.0,0.0,0.0,5621.0,63.0
2,568127133583555,-6.167469e+10,-3.460207e+10,-3.711834e+10,-3.696261e+10,-2.112113e+10,-5.360252e+10,-4.970846e+10,-5.230391e+10,-3.138140e+10,...,0,0,0,0,167.0,6.0,0.0,0.0,1826.0,39.0
3,561725390986555,-3.811280e+11,-2.634144e+11,-8.451206e+11,-8.405869e+11,-5.934585e+11,-2.220851e+11,-9.559368e+12,-1.594444e+12,-1.560891e+12,...,0,0,0,0,8.0,2.0,0.0,0.0,3352.0,28.0
4,575401131957555,9.366596e+12,2.939077e+12,6.061455e+12,2.116758e+13,7.450490e+12,5.746475e+12,3.431415e+12,4.151862e+12,4.373211e+12,...,1,0,1,0,109.0,169.0,0.0,15.0,3231.0,1152.0


In [409]:
dfinal_DJP = dfinal_faktur_riziq.merge(dfinal_scoring, on="NPWP", how = 'inner')
dfinal_DJP

,NPWP,FP_1,FP_2,FP_3,FP_4,FP_5,FP_6,FP_7,FP_8,FP_9,...,score_3,score_4,score_5,score_6,score_7,score_8,score_9,score_10,score_11,score_12
0,562197926987555,6.467929e+11,1.360289e+12,2.495005e+11,1.485149e+11,2.977032e+11,1.301476e+11,1.521944e+11,1.604439e+11,1.329168e+11,...,0.490598,0.446950,0.425040,0.466808,0.453214,0.449855,0.464601,0.470105,0.456958,0.493852
1,560540673980555,1.921724e+12,6.836181e+11,3.637846e+12,1.861282e+11,9.103713e+11,2.880675e+12,1.614540e+12,1.087428e+12,1.563684e+12,...,0.516716,0.519049,0.501951,0.488183,0.497002,0.502556,0.517378,0.494605,0.481987,0.505271
2,568127133583555,-6.167469e+10,-3.460207e+10,-3.711834e+10,-3.696261e+10,-2.112113e+10,-5.360252e+10,-4.970846e+10,-5.230391e+10,-3.138140e+10,...,0.443699,0.447433,0.484527,0.484561,0.507025,0.496808,0.452307,0.465099,0.465426,0.443524
3,561725390986555,-3.811280e+11,-2.634144e+11,-8.451206e+11,-8.405869e+11,-5.934585e+11,-2.220851e+11,-9.559368e+12,-1.594444e+12,-1.560891e+12,...,0.458823,0.433747,0.413993,0.493132,0.456861,0.432488,0.413526,0.492078,0.441298,0.519123
4,575401131957555,9.366596e+12,2.939077e+12,6.061455e+12,2.116758e+13,7.450490e+12,5.746475e+12,3.431415e+12,4.151862e+12,4.373211e+12,...,0.545899,0.541998,0.555817,0.509735,0.553797,0.536210,0.560412,0.512759,0.525773,0.510946
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26385,253148808580555,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.437306,0.000000,0.000000
26386,265046529568555,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,5.555500e+03,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000,0.000000,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
26387,574405549558555,0.000000e+00,0.000000e+00,3.111110e+05,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.598621,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
26388,574485675563555,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,6.045454e+05,0.000000e+00,0.000000e+00,...,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000


# Data untuk modelling siap di pakai

In [411]:
dfinal_DJP.to_csv('20210731_Data Pajak Per NPWP ready.csv')